In [239]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import math
from math import exp, sqrt

In [316]:
def generate_asset_price(S0,r,sigma,T,M,I):
    dt = float(T) / M
    paths = np.zeros((M + 1, I), np.float64)
    paths[0] = S0
    for t in range(1, M + 1):
        rand = np.random.standard_normal(I)
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * rand)
    return paths

In [317]:
S0 = 50.
K = 60.
r = 0.03
sigma = 0.20
T = 1
N = 12
deltat = T / N
I = 100000
discount_factor = np.exp(-r * T)

In [347]:
np.random.seed(123)
paths = generate_asset_price(S0, r, sigma, T, N, I)
paths[4]

array([46.90589987, 50.68183943, 43.75800015, ..., 50.93477649,
       51.88387489, 43.52883808])

In [348]:
#Plot the dataframe
#pd.DataFrame(paths).plot()
S_T = int(input("Please enter a number between 1 and 11 "))
paths[S_T]

Please enter a number between 1 and 11 1


array([47.00140179, 53.00774871, 50.86596877, ..., 53.50427726,
       57.09453792, 46.37432184])

In [349]:
#Payoff for a european put option
PutPayoff = np.maximum(0,K - paths[N])
time_step_back = (N-S_T)/N
print(time_step_back)
dis_payoff = np.exp(-r * time_step_back)*PutPayoff
price = discount_factor * (sum(PutPayoff)/float(I))
dis_payoff

0.9166666666666666


array([ 7.89184246, 16.32336283,  4.17479856, ...,  2.53206496,
       13.42444544, 24.13982223])

In [350]:
df_price = pd.DataFrame(paths[S_T], columns = ['Asset Price'])

In [351]:
y = np.array(dis_payoff)
x = np.array(df_price['Asset Price'])

In [352]:
x = x.reshape(-1,1)
print(x)
y = y.reshape(-1,1)
print(y)

[[47.00140179]
 [53.00774871]
 [50.86596877]
 ...
 [53.50427726]
 [57.09453792]
 [46.37432184]]
[[ 7.89184246]
 [16.32336283]
 [ 4.17479856]
 ...
 [ 2.53206496]
 [13.42444544]
 [24.13982223]]


In [353]:
# With train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 2)

In [362]:
from sklearn import preprocessing
#%% Standardization
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)
sc = preprocessing.StandardScaler().fit(Y_train)
Y_train = sc.transform(Y_train)
Y_test = sc.transform(Y_test)
Y_train

array([[ 0.77951611],
       [-1.02905726],
       [ 0.16508593],
       ...,
       [ 1.05952812],
       [-0.93044198],
       [-0.61450211]])

In [363]:
del scaler
del sc

In [364]:
df_Y_test = pd.DataFrame(Y_test, columns = ['Discounted Payoff'])

In [365]:
from sklearn.neural_network import MLPRegressor
# mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=400, alpha=1e-4,
#                     solver='sgd', verbose=10, tol=1e-4, random_state=1)
mlp = MLPRegressor(hidden_layer_sizes=(100,), alpha=1e-4,
                    solver='adam', activation = 'relu', verbose=0, tol=1e-4, random_state=9)

In [366]:
mlp.fit(X_train,Y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=9, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=0,
       warm_start=False)

In [367]:
Y_pred = mlp.predict(X_test)

In [368]:
#Root Mean Squared Error
import math
from sklearn.metrics import mean_squared_error, r2_score
trainScore = math.sqrt(mean_squared_error(Y_train, mlp.predict(X_train)))
testScore = math.sqrt(mean_squared_error(Y_test, Y_pred))
r2_train = r2_score(Y_train, mlp.predict(X_train))
r2_test = r2_score(Y_test, Y_pred)

#trainPredict = model.predict(X_train)
#testPredict = model.predict(X_test)
# calculate root mean squared error
#trainScore = math.sqrt(mean_squared_error(y_train, trainPredict))
#testScore = math.sqrt(mean_squared_error(y_test, testPredict))
print(trainScore)
print(testScore)
print(r2_train)
print(r2_test)

0.9603040490389061
0.9677508251726248
0.07781613339948212
0.07605426886533395


In [370]:
df_Y_pred = pd.DataFrame(Y_pred, columns = ['Discounted Payoff from Neural Net'])
test_vals = pd.concat([df_Y_test, df_Y_pred], axis=1)
test_vals.head(10)

,Discounted Payoff,Discounted Payoff from Neural Net
0,0.768148,-0.084745
1,-0.893699,0.207010
2,-0.012074,-0.349600
3,-0.140270,0.340829
4,-0.974382,0.069766
5,2.179483,0.209855
6,-1.248540,-0.078523
7,1.026685,0.371550
8,-0.362728,-0.662506
9,0.127390,0.201748
